In [ ]:
import guidance
from guidance import gen, select, models, assistant, system, user

In [ ]:
lm = models.Transformers("microsoft/Phi-3-mini-4k-instruct")

In [ ]:
chat_lm = lm

with user():
    chat_lm += "How are you?"

with assistant():
    chat_lm += gen("chat_response", max_tokens=20)

In [ ]:
chat_lm["chat_response"]

In [ ]:
food_lm = lm

with user():
    food_lm += "Do you like brussels sprouts?"

with assistant():
    food_lm += select(name="brussels", options=["Yes, I like them" , "No I despise them"])

In [ ]:
food_lm["brussels"]

In [ ]:
import json
from jsonschema import validate

In [ ]:
character_schema = """{
    "type": "object",
    "properties": {
        "description" : { "type" : "string" },
        "name" : { "type" : "string" },
        "age" : { "type" : "integer" },
        "armour" : { "type" : "string", "enum" : ["leather", "chainmail", "plate"] },
        "weapon" : { "type" : "string", "enum" : ["sword", "axe", "mace", "spear", "bow", "crossbow"] },
        "class" : { "type" : "string" },
        "mantra" : { "type" : "string" },
        "strength" : { "type" : "integer" },
        "quest_items" : { "type" : "array", "items" : { "type" : "string" } }
    },
    "additionalProperties": false
}
"""

character_schema_obj = json.loads(character_schema)

In [ ]:
from guidance import json as gen_json

In [ ]:
character_lm = lm

with system():
    character_lm += """You are a DM creating characters for a game set in a Tolkienesque setting.
Users will provide a one-line description of a character, you and should respond with a longer
description in JSON format.
"""

# Now give an example
with user():
    character_lm += "A quick and nimble fighter"

with assistant():
    character_lm += """{
    "description": "A quick and nimble fighter",
    "name": "Mokosh",
    "age": 20,
    "armour": "chainmail",
    "weapon": "sword",
    "class": "fighter",
    "mantra": "I am the sword of the gods",
    "strength": 10,
    "quest_items": [
        "Bag of holding",
        "Amulet of Perun",
    ]
}"""

# Now ask for our character
with user():
    character_lm += "A character attuned to the forest"

with assistant():
    character_lm += gen_json(schema=character_schema_obj, name="next_character", temperature=0.8)

In [ ]:
loaded_character = json.loads(character_lm["next_character"])

validate(instance=loaded_character, schema=character_schema_obj)

print(json.dumps(loaded_character, indent=4))

In [ ]:
@guidance
def generate_character(
    lm_curr,
    key: str,
    character_one_liner: str,
    temperature: float
):
    with system():
        lm_curr += """You are a DM creating characters for a game set in a Tolkienesque setting.
Users will provide a one-line description of a character, you and should respond with a longer
description in JSON format.
"""

    # Now give an example
    with user():
        lm_curr += "A quick and nimble fighter"

    with assistant():
        lm_curr += """{
    "description": "A quick and nimble fighter",
    "name": "Mokosh",
    "age": 20,
    "armour": "chainmail",
    "weapon": "sword",
    "class": "fighter",
    "mantra": "I am the sword of the gods",
    "strength": 10,
    "quest_items": [
        "Bag of holding",
        "Amulet of Perun",
    ]
}"""

    # Now ask for our character
    with user():
        lm_curr += character_one_liner

    with assistant():
        lm_curr += gen_json(schema=character_schema_obj, name=key)

    return lm_curr

In [ ]:
key = "new_character"
char_0 = lm + generate_character(character_one_liner="A crafty rogue", key=key, temperature=0.8)

print(json.dumps(json.loads(char_0[key]), indent=4))

In [ ]:
char_1 = char_0 + generate_character(character_one_liner="A paladin from strange lands", key=key, temperature=0.8)

print(json.dumps(json.loads(char_1[key]), indent=4))